# Server

In [1]:
import sc3nb as scn

The `SCServer` class is the central interface for 

* controlling the SuperCollider audio server process
* managing SuperCollider Objects
* using OSC for outgoing and incoming packets


To achieve all this the SCServer is

* registered as a client to the SuperCollider audio server process (scsynth) and exchanging [SuperCollider Commands](http://doc.sccode.org/Reference/Server-Command-Reference.html) with the server process
* and also running an OSC server in python which can communicate via OSC with scsynth and sclang.


For information about how to communicate using OSC see the [OSC communication notebook](../osc-communication-examples.ipynb). This notebook focuses on using the `SCServer` class for interacting with scsynth 

## Starting the Server

The most convienent way is to use the default sc3nb `SCServer` instance

In [2]:
sc = scn.startup()

<IPython.core.display.Javascript object>

Starting sclang process... [sclang | start reading ]


Done.
Registering OSC /return callback in sclang... Done.
Loading default sc3nb SynthDefs... Done.
Booting SuperCollider Server... [scsynth | start reading ]


Done.


In [3]:
sc.server

<SCServer addr=('127.0.0.1', 57110), process=<Process 'scsynth' (running) pid=12799>>

However you can also use the SCServer class directly

The server connection can be created

* locally using `boot`, which will start a scsynth process and connect to it or
* remote using `remote` for connecting to an already running scsynth process

In [4]:
serv = scn.SCServer()
serv

<SCServer (not booted)>

In [5]:
serv.boot()

Booting SuperCollider Server... 
SuperCollider Server port 57110 already used.
Trying to connect.
Done.


Notice how the `SCServer` always tries to boot using the default SuperCollider audio server port 57110.
But this port is already used by `sc.server` and thus the `SCServer` tries to connect to the already running instance using `SCserver.remote`. This enables a user to share the same scsynth instance with other users and/or use it from other notebooks. If the port to be used is explicitly specified the `SCServer` instance will fail instead of connecting.

The `SCServer` will register to the scsynth process using `SCServer.notify()`

Let's look how many clients are allowed and what the `client_id`s and the corresponding `default_group`s of the SCServer instances are.

In [6]:
print(f"The scsynth process of this SCServer instance allows {sc.server.max_logins} clients to login.")

The scsynth process of this SCServer instance allows 8 clients to login.


In [7]:
print(f"sc.server has client id {sc.server.client_id} and the default Group {sc.server.default_group}") 

sc.server has client id 1 and the default Group <Group(67108865) ~ {} children=[]>


In [8]:
print(f"serv has client id {serv.client_id} and the default Group {serv.default_group}") 

serv has client id 0 and the default Group <Group(1) s {} children=[]>


However also note that the instances use different ports meaning they are able to independendly send and receive OSC packets

In [9]:
sc.server.connection_info()

This instance is at ('127.0.0.1', 57131),
Known receivers: "scsynth" at ('127.0.0.1', 57110)
                 "sclang" at ('127.0.0.1', 57120)
                 


(('127.0.0.1', 57131),
 {('127.0.0.1', 57110): 'scsynth', ('127.0.0.1', 57120): 'sclang'})

and also note that `serv` is not connected to sclang but has the same connection info for scsynth.

In [10]:
serv.connection_info()

This instance is at ('127.0.0.1', 57133),
Known receivers: "scsynth" at ('127.0.0.1', 57110)
                 


(('127.0.0.1', 57133), {('127.0.0.1', 57110): 'scsynth'})

A Synth running on the SuperCollider audio server will be visible to all connected clients

In [11]:
serv_synth = scn.Synth("s2", {"amp": 0.05, "pan": -1, "freq": 100}, server=serv)

In [12]:
default_synth = scn.Synth("s2", {"amp": 0.05, "pan": 1, "freq": 440})  # no need to specify sc.server as server argument

This also includes sclang, which is another client of the scsynth process

In [13]:
%sc ~sclang_synth = Synth("s2", [\amp, 0.1])

-> Synth('s2' : 1000)


In [14]:
sc.server.dump_tree()

NODE TREE Group 0
   469762049 group
   402653185 group
   335544321 group
   268435457 group
   201326593 group
   134217729 group
   67108865 group
      20001 s2
        freq: 440 amp: 0.050000000745058 num: 4 pan: 1 lg: 0.10000000149012 gate: 1
   1 group
      1000 s2
        freq: 400 amp: 0.10000000149012 num: 4 pan: 0 lg: 0.10000000149012 gate: 1
      10001 s2
        freq: 100 amp: 0.050000000745058 num: 4 pan: -1 lg: 0.10000000149012 gate: 1


This also means freeing all Synths at once can be done with each client

In [15]:
sc.server.free_all()
# serv.free_all() 
# %sc s.freeAll

In [16]:
sc.server.dump_tree()

NODE TREE Group 0
   469762049 group
   402653185 group
   335544321 group
   268435457 group
   201326593 group
   134217729 group
   67108865 group
   1 group


and quitting one server also quits the others.

In [17]:
serv.quit()

Quitting SCServer... 

Done.


[scsynth | reached EOF ]


In [18]:
sc.server

<SCServer addr=('127.0.0.1', 57110), process=<Process 'scsynth' (died) pid=12799>>

Let's reboot the default server

In [19]:
sc.server.reboot()

Quitting SCServer... 

Done.
Booting SuperCollider Server... [scsynth | start reading ]
[scsynth]  Number of Devices: 2
[scsynth]     0 : "Built-in Microph"
[scsynth]     1 : "Built-in Output"
[scsynth]  
[scsynth]  "Built-in Microph" Input Device
[scsynth]     Streams: 1
[scsynth]        0  channels 2
[scsynth]  
[scsynth]  "Built-in Output" Output Device
[scsynth]     Streams: 1
[scsynth]        0  channels 2
[scsynth]  
[scsynth]  SC_AudioDriver: sample rate = 44100.000000, driver's block size = 512
[scsynth]  SuperCollider 3 server ready.


Done.


More information about [multi client setups](http://doc.sccode.org/Guides/MultiClient_Setups.html) can be found in the SuperCollider documentation.

## Configuring Server options

Startup options of the `SCServer` instance can be set via `ServerOptions`, which can be passed as argument when starting the `SCServer`

The default ServerOptions in sc3nb are:

In [20]:
scn.ServerOptions()

<ServerOptions ['-u', '57110', '-l', '6', '-i', '2', '-o', '2', '-a', '1024', '-c', '4096', '-b', '1024', '-R', '0']>

## Getting Information

The `SCServer` instance provides various kinds of information

* What nodes are currently running

In [21]:
sc.server.dump_tree()

[scsynth]  NODE TREE Group 0
[scsynth]     469762049 group
[scsynth]     402653185 group
[scsynth]     335544321 group
[scsynth]     268435457 group
[scsynth]     201326593 group
[scsynth]     134217729 group
[scsynth]     67108865 group
[scsynth]     1 group
NODE TREE Group 0
   469762049 group
   402653185 group
   335544321 group
   268435457 group
   201326593 group
   134217729 group
   67108865 group
   1 group


In [22]:
sc.server.query_tree()

Group(0)  {} children=[
  Group(469762049) s {} children=[],
  Group(402653185) s {} children=[],
  Group(335544321) s {} children=[],
  Group(268435457) s {} children=[],
  Group(201326593) s {} children=[],
  Group(134217729) s {} children=[],
  Group(67108865) s {} children=[],
  Group(1) ~ {} children=[]]

* The current status of the server, acquired via the `/status` OSC command

In [23]:
sc.server.status()

ServerStatus(num_ugens=6, num_synths=1, num_groups=9, num_synthdefs=7, avg_cpu=0.12140010297298431, peak_cpu=0.5158149600028992, nominal_sr=44100.0, actual_sr=44099.9999987347)

which can also be accessed directly via properties

In [24]:
sc.server.nominal_sr

44100.0

In [25]:
sc.server.num_synthdefs

7

* the version of the SC3 server process, acquired via the `/version` OSC command

In [26]:
sc.server.version()

ServerVersion(name='scsynth', major_version=3, minor_version=13, patch_version='.0', git_branch='Version-3.13.0', commit='3188503')

* the address of the SuperCollider audio server

In [27]:
sc.server.addr

('127.0.0.1', 57110)

* The connection info

In [28]:
sc.server.connection_info()

This instance is at ('127.0.0.1', 57131),
Known receivers: "scsynth" at ('127.0.0.1', 57110)
                 "sclang" at ('127.0.0.1', 57120)
                 


(('127.0.0.1', 57131),
 {('127.0.0.1', 57110): 'scsynth', ('127.0.0.1', 57120): 'sclang'})

* other runtime properties of the Server

In [29]:
sc.server.has_booted

True

In [30]:
sc.server.is_running

True

In [31]:
sc.server.client_id

1

In [32]:
sc.server.max_logins

8

In [33]:
sc.server.default_group

Group(67108865) s {} children=[]

In [34]:
sc.server.output_bus

Bus(rate='audio', ids=[0, 1])

In [35]:
sc.server.input_bus

Bus(rate='audio', ids=[2, 3])

## Controlling Volume

In [36]:
syn = scn.Synth("s2")

In [37]:
sc.server.volume

0.0

In [38]:
scn.dbamp(sc.server.volume)

1.0

In [39]:
sc.server.muted

False

In [40]:
sc.server.muted = True

In [41]:
sc.server.volume = -10.0

In [42]:
scn.dbamp(sc.server.volume)

0.31622776601683794

In [43]:
sc.server.muted = False

In [44]:
sc.server.volume = 0.0

In [45]:
scn.dbamp(sc.server.volume)

1.0

In [46]:
syn.free()
syn.wait(timeout=1)

## Server dumps

The Server process can dump information about

* incoming OSC packages. See console for output

In [47]:
sc.server.dump_osc() # specify level=0 to deactivate

* currently running Nodes 

In [48]:
sc.server.dump_tree()  # Notice how the OSC packet is now included in the output 

[scsynth]  [ "/g_dumpTree", 0, 1 ]
[scsynth]  NODE TREE Group 0
[scsynth]     -24 s1
[scsynth]       freq: 1000 amp: 0.050000000745058 num: 2 pan: 0 dur: 0.10000000149012 att: 0 curve: 42
[scsynth]     469762049 group
[scsynth]     402653185 group
[scsynth]     335544321 group
[scsynth]     268435457 group
[scsynth]     201326593 group
[scsynth]     134217729 group
[scsynth]     67108865 group
[scsynth]     20002 sc3nb_volumeAmpControl2
[scsynth]       volumeAmp: 0.31622776389122 volumeLag: 0.10000000149012 gate: 0 bus: 0
[scsynth]     1 group


[ "/g_dumpTree", 0, 1 ]
NODE TREE Group 0
   -24 s1
     freq: 1000 amp: 0.050000000745058 num: 2 pan: 0 dur: 0.10000000149012 att: 0 curve: 42
   469762049 group
   402653185 group
   335544321 group
   268435457 group
   201326593 group
   134217729 group
   67108865 group
   20002 sc3nb_volumeAmpControl2
     volumeAmp: 0.31622776389122 volumeLag: 0.10000000149012 gate: 0 bus: 0
   1 group


In [49]:
sc.server.blip() # see dumped bundle for test sound on console

[scsynth]  [ "#bundle", 16757651181134434304, 
[scsynth]    [ "#bundle", 16757651181134434304, 
[scsynth]      [ "/error", 0 ]
[scsynth]    ],


[scsynth]    [ "#bundle", 16757651181134434304, 
[scsynth]      [ "/s_new", "s1", -1, 0, 0, "freq", 500, "dur", 0.1, "num", 1 ]
[scsynth]    ],
[scsynth]    [ "#bundle", 16757651181993428992, 
[scsynth]      [ "/s_new", "s1", -1, 0, 0, "freq", 1000, "amp", 0.05, "num", 2, "att", 0, "dur", 0.1, "curve", 42 ]
[scsynth]    ],
[scsynth]    [ "#bundle", 16757651181993428992, 
[scsynth]      [ "/error", 1 ]
[scsynth]    ]
[scsynth]  ]


## Make a test sound

The following methods produces the `SCServer` startup sound.
The test sound should ease any anxiety whether the server is properly started/running

In [50]:
sc.server.blip()

[scsynth]  [ "#bundle", 16757651181185339392, 
[scsynth]    [ "#bundle", 16757651181185339392, 
[scsynth]      [ "/error", 0 ]
[scsynth]    ],
[scsynth]    [ "#bundle", 16757651181185339392, 
[scsynth]      [ "/s_new", "s1", -1, 0, 0, "freq", 500, "dur", 0.1, "num", 1 ]
[scsynth]    ],
[scsynth]    [ "#bundle", 16757651182044332032, 
[scsynth]      [ "/s_new", "s1", -1, 0, 0, "freq", 1000, "amp", 0.05, "num", 2, "att", 0, "dur", 0.1, "curve", 42 ]
[scsynth]    ],
[scsynth]    [ "#bundle", 16757651182044332032, 
[scsynth]      [ "/error", 1 ]
[scsynth]    ]
[scsynth]  ]


## Managing Nodes

* freeing all running nodes and reinitialize the server

In [51]:
sc.server.free_all()

[scsynth]  [ "/g_freeAll", 0 ]
[scsynth]  [ "/clearSched", ]
[scsynth]  [ "/g_new", 1, 0, 0 ]
[scsynth]  [ "/g_new", 67108865, 0, 0 ]
[scsynth]  [ "/g_new", 134217729, 0, 0 ]
[scsynth]  [ "/g_new", 201326593, 0, 0 ]
[scsynth]  [ "/g_new", 268435457, 0, 0 ]
[scsynth]  [ "/g_new", 335544321, 0, 0 ]
[scsynth]  [ "/g_new", 402653185, 0, 0 ]
[scsynth]  [ "/g_new", 469762049, 0, 0 ]
[scsynth]  [ "/g_new", 1, 0, 0 ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1007 ]
[scsynth]  ]
[scsynth]  [ "/d_recv", DATA[396] ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1008 ]
[scsynth]  ]
[scsynth]  [ "/sync", 5816 ]


In [52]:
sc.server.free_all(root=False)  # only frees the default group of this client

[scsynth]  [ "/g_freeAll", 67108865 ]
[scsynth]  [ "/clearSched", ]
[scsynth]  [ "/g_new", 67108865, 0, 0 ]
[scsynth]  [ "/g_new", 1, 0, 0 ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1009 ]
[scsynth]  ]
[scsynth]  [ "/notify", 1, 0 ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1010 ]
[scsynth]  ]
[scsynth]  [ "/d_recv", DATA[3037], 0 ]
[scsynth]  [ "/d_recv", DATA[1220], 0 ]
[scsynth]  [ "/d_recv", DATA[1285], 0 ]
[scsynth]  [ "/d_recv", DATA[396] ]
[scsynth]  [ "/d_recv", DATA[1120], 0 ]
[scsynth]  [ "/d_recv", DATA[2192], 0 ]
[scsynth]  [ "/d_recv", DATA[1135], 0 ]
[scsynth]  [ "/d_recv", DATA[190], 0 ]
[scsynth]  [ "/d_recv", DATA[1684], 0 ]
[scsynth]  [ "/d_recv", DATA[254], 0 ]
[scsynth]  [ "/d_recv", DATA[1685], 0 ]
[scsynth]  [ "/d_recv", DATA[1385], 0 ]
[scsynth]  [ "/d_recv", DATA[2290], 0 ]
[scsynth]  [ "/d_recv", DATA[1851], 0 ]
[scsynth]  [ "/d_recv", DATA[469], 0 ]
[scsynth]  [ "/d_recv", DATA[1444], 0 ]
[scsynth]  [ "/d_recv", DATA[619], 0 ]
[scsynth]  [

[scsynth]  [ "/d_recv", DATA[2797], 0 ]
[scsynth]  [ "/d_recv", DATA[1275], 0 ]
[scsynth]  [ "/d_recv", DATA[1436], 0 ]


[scsynth]  [ "/d_recv", DATA[1235], 0 ]


* send the `/clearSched` OSC command. This is automatically done when using `free_all`

In [53]:
sc.server.clear_schedule()

[scsynth]  [ "/d_recv", DATA[2459], 0 ]
[scsynth]  [ "/d_recv", DATA[1613], 0 ]
[scsynth]  [ "/d_recv", DATA[1335], 0 ]
[scsynth]  [ "/d_recv", DATA[1515], 0 ]
[scsynth]  [ "/d_recv", DATA[1170], 0 ]
[scsynth]  [ "/d_recv", DATA[2361], 0 ]
[scsynth]  [ "/d_recv", DATA[1339], 0 ]
[scsynth]  [ "/d_recv", DATA[1782], 0 ]
[scsynth]  [ "/clearSched", ]
[scsynth]  [ "/d_recv", DATA[937], 0 ]


[scsynth]  [ "/d_recv", DATA[1186], 0 ]
[scsynth]  [ "/d_recv", DATA[208], 0 ]
[scsynth]  [ "/d_recv", DATA[669], 0 ]
[scsynth]  [ "/d_recv", DATA[1353], 0 ]
[scsynth]  [ "/d_recv", DATA[501], 0 ]


* Execute init hooks. This is also automatically done when using `free_all`, `init` or `connect_sclang`

In [54]:
sc.server.execute_init_hooks()

[scsynth]  [ "/d_recv", DATA[2868], 0 ]
[scsynth]  [ "/d_recv", DATA[263], 0 ]
[scsynth]  [ "/d_recv", DATA[605], 0 ]
[scsynth]  [ "/d_recv", DATA[1034], 0 ]
[scsynth]  [ "/d_recv", DATA[519], 0 ]
[scsynth]  [ "/d_recv", DATA[1853], 0 ]
[scsynth]  [ "/d_recv", DATA[1952], 0 ]
[scsynth]  [ "/d_recv", DATA[281], 0 ]
[scsynth]  [ "/d_recv", DATA[1117], 0 ]
[scsynth]  [ "/d_recv", DATA[2023], 0 ]
[scsynth]  [ "/d_recv", DATA[1055], 0 ]


[scsynth]  [ "/d_recv", DATA[964], 0 ]


[scsynth]  [ "/d_recv", DATA[934], 0 ]
[scsynth]  [ "/d_recv", DATA[1106], 0 ]
[scsynth]  [ "/d_recv", DATA[430], 0 ]
[scsynth]  [ "/d_recv", DATA[734], 0 ]
[scsynth]  [ "/d_recv", DATA[970], 0 ]
[scsynth]  [ "/d_recv", DATA[768], 0 ]
[scsynth]  [ "/d_recv", DATA[235], 0 ]
[scsynth]  [ "/d_recv", DATA[834], 0 ]
[scsynth]  [ "/d_recv", DATA[2628], 0 ]
[scsynth]  [ "/d_recv", DATA[1768], 0 ]
[scsynth]  [ "/d_recv", DATA[2966], 0 ]
[scsynth]  [ "/d_recv", DATA[854], 0 ]
[scsynth]  [ "/d_recv", DATA[1008], 0 ]


[scsynth]  [ "/d_recv", DATA[2530], 0 ]
[scsynth]  [ "/d_recv", DATA[245], 0 ]
[scsynth]  [ "/d_recv", DATA[1177], 0 ]
[scsynth]  [ "/d_recv", DATA[1602], 0 ]
[scsynth]  [ "/d_recv", DATA[1277], 0 ]
[scsynth]  [ "/d_recv", DATA[396], 0 ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1011 ]
[scsynth]  ]
[scsynth]  [ "/d_recv", DATA[390], 0 ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1012 ]
[scsynth]  ]
[scsynth]  [ "/g_new", 1, 0, 0 ]
[scsynth]  [ "/g_new", 67108865, 0, 0 ]
[scsynth]  [ "/g_new", 134217729, 0, 0 ]
[scsynth]  [ "/g_new", 201326593, 0, 0 ]
[scsynth]  [ "/g_new", 268435457, 0, 0 ]
[scsynth]  [ "/g_new", 335544321, 0, 0 ]
[scsynth]  [ "/g_new", 402653185, 0, 0 ]
[scsynth]  [ "/g_new", 469762049, 0, 0 ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1013 ]
[scsynth]  ]


[scsynth]  [ "/g_new", 1, 0, 0 ]
[scsynth]  [ "/g_new", 67108865, 0, 0 ]
[scsynth]  [ "/g_new", 134217729, 0, 0 ]
[scsynth]  [ "/g_new", 201326593, 0, 0 ]
[scsynth]  [ "/g_new", 268435457, 0, 0 ]
[scsynth]  [ "/g_new", 335544321, 0, 0 ]
[scsynth]  [ "/g_new", 402653185, 0, 0 ]
[scsynth]  [ "/g_new", 469762049, 0, 0 ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1014 ]
[scsynth]  ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1015 ]
[scsynth]  ]
[scsynth]  [ "/d_recv", DATA[396] ]
[scsynth]  [ "#bundle", 1, 
[scsynth]    [ "/sync", 1016 ]
[scsynth]  ]


* Adding init hooks. 

In [55]:
sc.server.send_default_groups

<bound method SCServer.send_default_groups of <SCServer addr=('127.0.0.1', 57110), process=<Process 'scsynth' (running) pid=12807>>>

* Syncing the SuperCollider audio server by sending a `/sync` OSC command and waiting for the reply.

In [56]:
sc.server.sync()

[scsynth]  [ "/sync", 7963 ]


True

## Allocating IDs

The `SCServer` instance manages the IDs for Nodes, Buffers and Buses for the SuperCollider Objects via the following methods.
These can also be used for getting suitable IDs when manually creating OSC packages.

* Get the IDs via the allocator.

In [57]:
ids = sc.server.buffer_ids.allocate(num=2)
ids

[128, 129]

* Free the IDs after usage via the allocator.

In [58]:
sc.server.buffer_ids.free(ids)

There are allocators for

* Nodes - `sc.server.node_ids`
* Buffer - `sc.server.buffer_ids`
* Buses (Audio and Control) - `sc.server.audio_bus_ids`, `sc.server.control_bus_ids`

In [59]:
sc.server.reboot()

Quitting SCServer... [scsynth]  [ "/quit", ]


[scsynth | reached EOF ]
Done.
Booting SuperCollider Server... 

[scsynth | start reading ]


[scsynth]  Number of Devices: 2
[scsynth]     0 : "Built-in Microph"
[scsynth]     1 : "Built-in Output"
[scsynth]  
[scsynth]  "Built-in Microph" Input Device
[scsynth]     Streams: 1
[scsynth]        0  channels 2
[scsynth]  
[scsynth]  "Built-in Output" Output Device
[scsynth]     Streams: 1
[scsynth]        0  channels 2
[scsynth]  
[scsynth]  SC_AudioDriver: sample rate = 44100.000000, driver's block size = 512
[scsynth]  SuperCollider 3 server ready.


Done.


In [60]:
# example to see how consecutive buffer alloc works:
ids = sc.server.buffer_ids.allocate(num=5)
print("5 buffers:", ids)
sc.server.buffer_ids.free(ids[0:2])
print("freed buffers ", ids[0:2])
ids4 = sc.server.buffer_ids.allocate(num=4)
print("allocated 4 buffers:", ids4, "-> new numbers to be consecutive")
sc.server.sync()
ids2 = sc.server.buffer_ids.allocate(num=2)
print("allocated 2 buffers:", ids2, "-> using the two freed before")

5 buffers: [128, 129, 130, 131, 132]
freed buffers  [128, 129]
allocated 4 buffers: [133, 134, 135, 136] -> new numbers to be consecutive


allocated 2 buffers: [128, 129] -> using the two freed before


## Handling SynthDefs

The server offers the following methods for handling SynthDefs.
These are shortcuts for the respective SynthDef methods.

```
sc.server.send_synthdef
sc.server.load_synthdef
sc.server.load_synthdefs
```
Refer to the [SynthDef guide](synthdef-examples.ipynb) for more information about SynthDefs.

In [61]:
sc.exit()

Quitting SCServer... [scsynth | reached EOF ]
Done.
Exiting sclang... [sclang | reached EOF ]
Done.
